# Exploratory Data Analysis by SQL
<p> SQL techniques (Join, Cast, Aggregation and CTEs) are used in this notebook.</p>

## Key Findings
* There is a strong positive correlation between customer reviews and revenue. The company should start several initiatvies such as free sample and coupon to encourage customers to write comments more about their purchases.
<br />

* In terms of brand, the website currently sell 2575 Adidas products and 545 Nike products. Total revenue of Adidas (11.5k) is 14 times of Nike (0.8k). The better sales of Adidas list (avg. 4.5k per list) over Nike list (avg. 1.5k per list) is likely due to different price perception drove by different promotion strategy, considering Adidas products are on promotion by 33% discount and Nike products didn't run any promotion on the website.
<br />

* There are currently 3179 product listings on the site with 2928 products have traffic. The 251 long-tail selection without traffic could be removed from inventory to optimize free cash flow.
<br />

* 2,700 of the company's lists are footwear, accounting for 85% of the company's selection and generated an average revenue of $3,118 per product, 6 times that of clothing products.

In [1]:
%%sql
postgresql:///sports

SELECT COUNT(*) AS total_rows, 
    COUNT(i.description) AS count_description, 
    COUNT(f.listing_price) AS count_listing_price, 
    COUNT(t.last_visited) AS count_last_visited 
FROM info AS i
INNER JOIN finance AS f
    ON i.product_id = f.product_id
INNER JOIN traffic AS t
    ON t.product_id = f.product_id;

UsageError: Cell magic `%%sql` not found.


In [17]:
%%sql

WITH footwear AS
(
    SELECT i.description, f.revenue
    FROM info AS i
    INNER JOIN finance AS f 
        ON i.product_id = f.product_id
    WHERE i.description ILIKE '%shoe%'
        OR i.description ILIKE '%trainer%'
        OR i.description ILIKE '%foot%'
        AND i.description IS NOT NULL
)

SELECT COUNT(*) AS num_footwear_products, 
    percentile_disc(0.5) WITHIN GROUP (ORDER BY revenue) AS average_footwear_revenue
FROM footwear;

 * postgresql:///sports
1 rows affected.


num_footwear_products,average_footwear_revenue
2700,3118.36


In [16]:
%%sql

WITH footwear AS
(
SELECT i.description, f.revenue
FROM info AS i
INNER JOIN finance AS f 
ON i.product_id = f.product_id
WHERE i.description ILIKE '%shoe%'
OR i.description ILIKE '%trainer%'
OR i.description ILIKE '%foot%'
AND i.description IS NOT NULL
)

SELECT COUNT(i.*) AS num_clothing_products, 
    percentile_disc(0.5) WITHIN GROUP (ORDER BY f.revenue) AS average_clothing_revenue
FROM info AS i
INNER JOIN finance AS f on i.product_id = f.product_id
WHERE i.description NOT IN (SELECT description FROM footwear);


 * postgresql:///sports
1 rows affected.


num_clothing_products,average_clothing_revenue
417,503.82


In [9]:
%%sql

SELECT b.brand, COUNT(f.*), CAST(SUM(f.revenue) as DECIMAL(18,0)) as total_revenue
FROM finance AS f
INNER JOIN brands AS b 
ON f.product_id = b.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand
ORDER BY total_revenue DESC;

 * postgresql:///sports
2 rows affected.


brand,count,total_revenue
Adidas,2575,11526619
Nike,545,802283


In [5]:
%%sql

SELECT b.brand, f.listing_price::integer, COUNT(f.*)
FROM finance AS f
INNER JOIN brands AS b 
    ON f.product_id = b.product_id
WHERE listing_price > 0
GROUP BY b.brand, f.listing_price
ORDER BY listing_price DESC;

 * postgresql:///sports
77 rows affected.


brand,listing_price,count
Adidas,300,2
Adidas,280,4
Adidas,240,5
Adidas,230,8
Adidas,220,11
Nike,200,1
Adidas,200,8
Nike,190,2
Adidas,190,7
Nike,180,4


In [10]:
%%sql

SELECT b.brand, COUNT(f.*), CAST(SUM(f.revenue) as DECIMAL(18,0)) as total_revenue,
CASE WHEN f.listing_price < 50 THEN 'Budget'
    WHEN f.listing_price >= 50 AND f.listing_price < 90 THEN 'Average'
    WHEN f.listing_price >= 90 AND f.listing_price < 150 THEN 'Expensive'
    ELSE 'Elite' END AS price_category
FROM finance AS f
INNER JOIN brands AS b 
    ON f.product_id = b.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand, price_category
ORDER BY total_revenue DESC;

 * postgresql:///sports
8 rows affected.


brand,count,total_revenue,price_category
Adidas,1189,4852192,Average
Adidas,453,3290185,Expensive
Adidas,799,1800693,Budget
Adidas,134,1583550,Elite
Nike,366,600908,Budget
Nike,78,107632,Expensive
Nike,52,70136,Elite
Nike,49,23607,Average


In [12]:
%%sql
    
SELECT b.brand,CAST(AVG(f.discount) as DECIMAL(18,2)) * 100 AS average_discount
FROM brands AS b
INNER JOIN finance AS f 
ON b.product_id = f.product_id
GROUP BY b.brand
HAVING b.brand IS NOT NULL
ORDER BY average_discount;

 * postgresql:///sports
2 rows affected.


brand,average_discount
Nike,0.00
Adidas,33.00


In [18]:
%%sql

SELECT CAST(CORR(r.reviews, f.revenue) as DECIMAL(18,2)) AS review_revenue_corr
FROM reviews AS r
INNER JOIN finance AS f 
ON r.product_id = f.product_id;

 * postgresql:///sports
1 rows affected.


review_revenue_corr
0.65


In [5]:
%%sql

SELECT b.brand, DATE_PART('month', t.last_visited) AS month, COUNT(r.*) AS num_reviews
FROM brands AS b
INNER JOIN traffic AS t 
    ON b.product_id = t.product_id
INNER JOIN reviews AS r 
    ON t.product_id = r.product_id
GROUP BY b.brand, month
HAVING b.brand IS NOT NULL
    AND DATE_PART('month', t.last_visited) IS NOT NULL
ORDER BY b.brand, month;

 * postgresql:///sports
24 rows affected.


brand,month,num_reviews,total_order
Adidas,1.0,253,253
Adidas,2.0,272,272
Adidas,3.0,269,269
Adidas,4.0,180,180
Adidas,5.0,172,172
Adidas,6.0,159,159
Adidas,7.0,170,170
Adidas,8.0,189,189
Adidas,9.0,181,181
Adidas,10.0,192,192
